## Author: Blaine Hill

In this notebook, we train a score matching model as described by [Yang Song](https://yang-song.net/blog/2021/score/)


$$s_\theta(x) = \nabla_x \log p_\theta(x) = -\nabla_x f_\theta(x) - \nabla_x \log Z_\theta = -\nabla_x f_\theta(x)$$

In [1]:
#TODO add probability flow ode and use sde for noise

#TODO add wandb functionality

#TODO abstract any dimensions 

#TODO add conditional generation with annealed langevin sampling (essentially the sde)

In [2]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from torch_geometric.nn import RotatE
# from torch_geometric.datasets import FB15k_237
# import sys
# import os
# import os.path as osp
# import math
# import numpy as np
# from icecream import ic

# # Set up paths
# parent_dir = os.path.abspath(os.pardir)
# sys.path.append(parent_dir)
# data_path = osp.join(parent_dir, "data", "FB15k_237")

# # Load dataset
# device = "cuda" if torch.cuda.is_available() else "cpu"
# train_data = FB15k_237(data_path, split="train")[0].to(device)
# val_data = FB15k_237(data_path, split="val")[0].to(device)
# test_data = FB15k_237(data_path, split="test")[0].to(device)

# # Load pre-trained RotatE model
# model = RotatE(num_nodes=train_data.num_nodes, num_relations=train_data.num_edge_types, hidden_channels=50).to(device)
# model.load_state_dict(torch.load("../embedding_model/FB15k_237_RotatE_embedding_model_weights.pth"))
# model.eval()

# # Get entity and relation embeddings
# with torch.no_grad():
#     entity_embeddings_real = model.node_emb.weight.detach()
#     entity_embeddings_im = model.node_emb_im.weight.detach()
#     entity_embeddings = torch.cat([entity_embeddings_real, entity_embeddings_im], dim=-1)

#     relation_embeddings = model.rel_emb.weight.detach()

# ic(entity_embeddings_real.shape)
# ic(entity_embeddings_im.shape)
# ic(entity_embeddings.shape)

# ic(relation_embeddings.shape)




# class ScoreModel(nn.Module):
#     def __init__(self, embedding_dim):
#         super(ScoreModel, self).__init__()
#         self.net = nn.Sequential(
#             nn.Linear(embedding_dim * 2 + 50, 512),  # Assuming concatenation of h, r, and t embeddings
#             nn.ReLU(),
#             nn.Linear(512, 256),
#             nn.ReLU(),
#             nn.Linear(256, 1)  # Outputting a single score
#         )

#     def forward(self, h, r, t):
#         x = torch.cat([h, r, t], dim=-1)
#         return self.net(x)

# def sde(h, r, t, beta):
#     # This is a simplified SDE for demonstration. You might need a more complex one.
#     drift = score_function(h, r, t)  # Assuming score_function is defined
#     diffusion = beta  # This could be a function of time or constant
#     return drift, diffusion

# def rsde(h, r, t, beta):
#     drift, _ = sde(h, r, t, beta)
#     reverse_drift = -drift + 2 * diffusion  # Simplified reverse drift
#     return reverse_drift, diffusion

# def annealed_langevin_sampling(h, r, t, beta_schedule, num_steps=1000):
#     samples = torch.randn_like(h)  # Initialize with noise
#     for beta in beta_schedule:
#         drift, diffusion = rsde(h, r, t, beta)
#         samples += drift + torch.sqrt(diffusion) * torch.randn_like(samples)
#     return samples

# beta_schedule = torch.linspace(start=0.1, end=20, steps=num_steps)

# optimizer = optim.Adam(score_model.parameters(), lr=1e-3)

# for epoch in range(num_epochs):
#     for h, r, t in data_loader:  # Assuming data_loader is defined
#         optimizer.zero_grad()
#         scores = score_model(h, r, t)
#         loss = compute_loss(scores)  # Define compute_loss based on your specific task
#         loss.backward()
#         optimizer.step()

#     # Evaluate on link prediction tasks
# def compute_metrics(data, entity_embeddings, relation_embeddings):
#     mean_rank = []
#     mean_reciprocal_rank = []
#     hits_at_k = {k: [] for k in [1, 3, 10]}

#     for h, r, t in data:
#         scores = []
#         for candidate_tail in range(train_data.num_nodes):
#             score = score_function(entity_embeddings[h], relation_embeddings[r], entity_embeddings[candidate_tail])
#             scores.append(score)

#         true_score = scores[t]
#         ranks = 1 + torch.argsort(torch.tensor(scores), descending=True).tolist().index(t)
#         mean_rank.append(ranks)
#         mean_reciprocal_rank.append(1 / ranks)

#         for k in hits_at_k:
#             if ranks <= k:
#                 hits_at_k[k].append(1)
#             else:   
#                 hits_at_k[k].append(0)

#     mean_rank = np.mean(mean_rank)
#     mean_reciprocal_rank = np.mean(mean_reciprocal_rank)
#     hits_at_k = {k: np.mean(v) for k, v in hits_at_k.items()}

#     return mean_rank, mean_reciprocal_rank, hits_at_k

# # Compute metrics for the original embeddings
# original_mean_rank, original_mean_reciprocal_rank, original_hits_at_k = compute_metrics(test_data, entity_embeddings, model.relation_emb.weight)
# print("Original Embeddings:")
# print(f"Mean Rank: {original_mean_rank}")
# print(f"Mean Reciprocal Rank: {original_mean_reciprocal_rank}")
# print(f"Hits@1: {original_hits_at_k[1]}, Hits@3: {original_hits_at_k[3]}, Hits@10: {original_hits_at_k[10]}")

# # Compute metrics for the conditioned embeddings
# conditioned_mean_rank, conditioned_mean_reciprocal_rank, conditioned_hits_at_k = compute_metrics(test_data, entity_embeddings_conditioned, model.relation_emb.weight)
# print("\nConditioned Embeddings:")
# print(f"Mean Rank: {conditioned_mean_rank}")
# print(f"Mean Reciprocal Rank: {conditioned_mean_reciprocal_rank}")
# print(f"Hits@1: {conditioned_hits_at_k[1]}, Hits@3: {conditioned_hits_at_k[3]}, Hits@10: {conditioned_hits_at_k[10]}")

In [3]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import Adam
# from torch.utils.data import DataLoader, TensorDataset
# from torch_geometric.nn import RotatE
# from torch_geometric.datasets import FB15k_237
# import sys
# import os
# import os.path as osp
# import math
# import numpy as np
# from icecream import ic

# # Set up paths
# parent_dir = os.path.abspath(os.pardir)
# sys.path.append(parent_dir)
# data_path = osp.join(parent_dir, "data", "FB15k_237")

# # Load dataset
# device = "cuda" if torch.cuda.is_available() else "cpu"
# train_data = FB15k_237(data_path, split="train")[0].to(device)
# val_data = FB15k_237(data_path, split="val")[0].to(device)
# test_data = FB15k_237(data_path, split="test")[0].to(device)


# # Define cosine noise schedule
# def cosine_beta_schedule(timesteps, s=0.008):
#     """
#     Cosine schedule as proposed in https://openreview.net/forum?id=-NEXDKk8gZ
#     """
#     steps = timesteps + 1
#     x = torch.linspace(0, timesteps, steps)
#     alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
#     alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
#     betas = 1 - alphas_cumprod[1:] / alphas_cumprod[:-1]
#     return torch.clip(betas, 0, 0.999)
    


# class ScoreModel(nn.Module):
#     def __init__(self, embedding_model, embedding_dim, relation_dim):
#         super(ScoreModel, self).__init__()
#         self.embedding_model = embedding_model
#         self.embedding_dim = embedding_dim
#         self.relation_dim = relation_dim
#         self.score_net = nn.Sequential(
#             nn.Linear(embedding_dim + relation_dim, 512, dtype=torch.float),
#             nn.ReLU(),
#             nn.Linear(512, 512, dtype=torch.float),
#             nn.ReLU(),
#             nn.Linear(512, 1, dtype=torch.float)
#         )

#     def forward(self, h, r, t, noise_level=None):
#         h_emb = self.embedding_model.node_emb(h)
#         r_emb = self.embedding_model.rel_emb(r)
#         t_emb = self.embedding_model.node_emb(t)

#         if noise_level is not None:
#             noise_h = torch.randn_like(h_emb) * noise_level
#             noise_r = torch.randn_like(r_emb) * noise_level
#             noise_t = torch.randn_like(t_emb) * noise_level
#             h_emb = h_emb + noise_h
#             r_emb = r_emb + noise_r
#             t_emb = t_emb + noise_t

#         input = torch.cat([h_emb, r_emb, t_emb], dim=-1)
#         score = self.score_net(input)
#         return score

# def build_model(config):
#     # Load pre-trained RotatE model (embedding_model)
#     embedding_model = RotatE(num_nodes=train_data.num_nodes, num_relations=train_data.num_edge_types, hidden_channels=50).to(config["device"])
#     embedding_model.load_state_dict(torch.load("../embedding_model/FB15k_237_RotatE_embedding_model_weights.pth"))
#     embedding_model.eval()

#     # Get entity and relation embeddings
#     with torch.no_grad():
#         entity_embeddings_real = embedding_model.node_emb.weight.detach()
#         entity_embeddings_im = embedding_model.node_emb_im.weight.detach()
#         entity_embeddings = torch.cat([entity_embeddings_real, entity_embeddings_im], dim=-1)
#         relation_embeddings = embedding_model.rel_emb.weight.detach()

#     # Initialize score-based model
#     score_model = ScoreModel(embedding_model, entity_embeddings.shape[-1], relation_embeddings.shape[-1]).to(config["device"])


#     # Create data loaders
#     train_dataset = TensorDataset(train_data.edge_index[0], train_data.edge_type, train_data.edge_index[1])
#     train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)

#     val_dataset = TensorDataset(val_data.edge_index[0], val_data.edge_type, val_data.edge_index[1])
#     val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

#     test_dataset = TensorDataset(test_data.edge_index[0], test_data.edge_type, test_data.edge_index[1])
#     test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

#     # Initialize optimizer
#     optimizer = Adam(score_model.parameters(), lr=config["lr"])


#     return score_model, optimizer, train_loader, val_loader, test_loader


# # Define loss function
# def denoising_score_matching_loss(score_model, h, r, t, noise_level):
#     """
#     Denoising score-matching loss with noise-conditional score networks.
#     """
#     true_score = score_model(h, r, t) #simply do not pass in the noise
#     noisy_score = score_model(h, r, t, noise_level)
#     return ((true_score - noisy_score) ** 2).mean()



# def train_model(score_model, optimizer, train_loader, config):
#     score_model.train()
#     loss_epoch = 0
#     for batch in train_loader:
#         h, r, t = batch
#         h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])

#         optimizer.zero_grad()
#         noise_level = torch.rand(1) * config["noise_max"]  # Sample a noise level for this batch
#         loss = denoising_score_matching_loss(score_model, h, r, t, noise_level)
#         loss.backward()
#         optimizer.step()

#         loss_epoch += loss.item()

#     return loss_epoch / len(train_loader)

# def test_model(score_model, test_loader, config):
#     score_model.eval()
#     test_loss = 0
#     with torch.no_grad():
#         for batch in test_loader:
#             h, r, t = batch
#             h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])
#             loss = denoising_score_matching_loss(score_model, h, r, t, None)
#             test_loss += loss.item()

#     return test_loss / len(test_loader)

# def compute_metrics(score_model, test_loader):
#     """
#     Compute link prediction metrics: mean rank, mean reciprocal rank, and Hits@k.
#     """
#     pass

# def main(config):
#     score_model, optimizer, train_loader, val_loader, test_loader = build_model(config)

#     for epoch in range(config["num_epochs"]):
#         train_loss = train_model(score_model, optimizer, train_loader, config)
#         test_loss = test_model(score_model, test_loader, config)
#         print(f"\n\nEpoch {epoch + 1}\nTrain Loss: {train_loss:.20f}\nTest Loss: {test_loss:.20f}")

#     compute_metrics(score_model, test_loader)

# config = {
#     "batch_size": 64,
#     "num_epochs": 2,
#     "num_steps": 1000,
#     "lr": 1e-4,
#     "device": device,
#     "noise_max": 0.5  # 20% of the original data values
# }

# main(config)

In [4]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import Adam
# from torch.utils.data import DataLoader, TensorDataset
# from torch_geometric.nn import RotatE
# from torch_geometric.datasets import FB15k_237
# import sys
# import os
# import os.path as osp
# import math
# import numpy as np
# from icecream import ic

# # Set up paths
# parent_dir = os.path.abspath(os.pardir)
# sys.path.append(parent_dir)
# data_path = osp.join(parent_dir, "data", "FB15k_237")

# # Load dataset
# device = "cuda" if torch.cuda.is_available() else "cpu"
# train_data = FB15k_237(data_path, split="train")[0].to(device)
# val_data = FB15k_237(data_path, split="val")[0].to(device)
# test_data = FB15k_237(data_path, split="test")[0].to(device)


# # Define cosine noise schedule
# def cosine_beta_schedule(timesteps, s=0.008):
#     """
#     Cosine schedule as proposed in https://openreview.net/forum?id=-NEXDKk8gZ
#     """
#     steps = timesteps + 1
#     x = torch.linspace(0, timesteps, steps)
#     alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
#     alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
#     betas = 1 - alphas_cumprod[1:] / alphas_cumprod[:-1]
#     return torch.clip(betas, 0, 0.999)

# def diffusion_process(e, r, t, beta, device):
#     """
#     Diffusion process for triplets.
#     e, r, t: entity and relation embeddings
#     beta: noise level
#     """
#     noise_e = torch.randn_like(e, device=device)
#     noise_r = torch.randn_like(r, device=device)
#     noise_t = torch.randn_like(t, device=device)
    
#     noisy_e = (1 - beta) ** 0.5 * e + beta ** 0.5 * noise_e
#     noisy_r = (1 - beta) ** 0.5 * r + beta ** 0.5 * noise_r
#     noisy_t = (1 - beta) ** 0.5 * t + beta ** 0.5 * noise_t
    
#     return noisy_e, noisy_r, noisy_t 


# class ScoreModel(nn.Module):
#     def __init__(self, embedding_model, embedding_dim, relation_dim, time_emb_dim):
#         super(ScoreModel, self).__init__()
#         self.embedding_model = embedding_model
#         self.embedding_dim = embedding_dim
#         self.relation_dim = relation_dim
#         self.time_emb = nn.Linear(1, time_emb_dim)
#         self.score_net = nn.Sequential(
#             nn.Linear(embedding_dim + relation_dim + time_emb_dim, 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, embedding_dim + relation_dim)
#         )

#     def forward(self, h, r, t, t_emb):
#         h_emb = self.embedding_model.node_emb(h)
#         r_emb = self.embedding_model.rel_emb(r)
#         t_emb = self.embedding_model.node_emb(t)
#         time_emb = self.time_emb(t_emb)
#         input = torch.cat([h_emb, r_emb, t_emb, time_emb], dim=-1)
#         score = self.score_net(input)
#         return score

# def build_model(config):
#     # Load pre-trained RotatE model (embedding_model)
#     embedding_model = RotatE(num_nodes=train_data.num_nodes, num_relations=train_data.num_edge_types, hidden_channels=50).to(config["device"])
#     embedding_model.load_state_dict(torch.load("../embedding_model/FB15k_237_RotatE_embedding_model_weights.pth"))
#     embedding_model.eval()

#     # Get entity and relation embeddings
#     with torch.no_grad():
#         entity_embeddings_real = embedding_model.node_emb.weight.detach()
#         entity_embeddings_im = embedding_model.node_emb_im.weight.detach()
#         entity_embeddings = torch.cat([entity_embeddings_real, entity_embeddings_im], dim=-1)
#         relation_embeddings = embedding_model.rel_emb.weight.detach()

#     # Initialize score-based model
#     score_model = ScoreModel(embedding_model, entity_embeddings.shape[-1], relation_embeddings.shape[-1]).to(config["device"])


#     # Create data loaders
#     train_dataset = TensorDataset(train_data.edge_index[0], train_data.edge_type, train_data.edge_index[1])
#     train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)

#     val_dataset = TensorDataset(val_data.edge_index[0], val_data.edge_type, val_data.edge_index[1])
#     val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

#     test_dataset = TensorDataset(test_data.edge_index[0], test_data.edge_type, test_data.edge_index[1])
#     test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

#     # Initialize optimizer
#     optimizer = Adam(score_model.parameters(), lr=config["lr"])


#     return score_model, optimizer, train_loader, val_loader, test_loader


# def denoising_score_matching_loss(score_model, h, r, t, noisy_h, noisy_r, noisy_t, t_emb):
#     """
#     Denoising score-matching loss for triplets.
#     """
#     true_score = score_model(h, r, t, t_emb)
#     noisy_score = score_model(noisy_h, noisy_r, noisy_t, t_emb)
#     return ((true_score - noisy_score) ** 2).mean()



# def train_model(score_model, optimizer, train_loader, config):
#     score_model.train()
#     loss_epoch = 0
#     for batch in train_loader:
#         h, r, t = batch
#         h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])

#         optimizer.zero_grad()
#         beta = torch.rand(1, device=config["device"]) * config["noise_max"]  # Sample a noise level for this batch
#         t_emb = torch.tensor([beta], dtype=torch.float, device=config["device"])
#         noisy_h, noisy_r, noisy_t = diffusion_process(h, r, t, beta, config["device"])
#         loss = denoising_score_matching_loss(score_model, h, r, t, noisy_h, noisy_r, noisy_t, t_emb)
#         loss.backward()
#         optimizer.step()

#         loss_epoch += loss.item()

#     return loss_epoch / len(train_loader)

# def test_model(score_model, test_loader, config):
#     score_model.eval()
#     test_loss = 0
#     with torch.no_grad():
#         for batch in test_loader:
#             h, r, t = batch
#             h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])
#             t_emb = torch.tensor([0.0], dtype=torch.float, device=config["device"])
#             loss = denoising_score_matching_loss(score_model, h, r, t, h, r, t, t_emb)
#             test_loss += loss.item()

#     return test_loss / len(test_loader)

# def compute_metrics(score_model, test_loader, config, k=10):
#     score_model.eval()
#     mean_rank = []
#     mean_reciprocal_rank = []
#     hits_at_k = []
#     with torch.no_grad():
#         for batch in test_loader:
#             h, r, t = batch
#             h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])
#             t_emb = torch.tensor([0.0], dtype=torch.float, device=config["device"])
#             scores = score_model(h, r, t, t_emb)
            
#             # Compute metrics
#             ranks = torch.argsort(scores, dim=-1, descending=True) + 1
#             mean_rank.append(ranks.float().mean().item())
#             mean_reciprocal_rank.append((1 / ranks.float()).mean().item())
#             hits_at_k.append((ranks <= k).float().mean().item())
    
#     print(f"Mean Rank: {sum(mean_rank) / len(mean_rank):.4f}")
#     print(f"Mean Reciprocal Rank: {sum(mean_reciprocal_rank) / len(mean_reciprocal_rank):.4f}")
#     print(f"Hits@{k}: {sum(hits_at_k) / len(hits_at_k):.4f}")

# def link_prediction(score_model, test_loader, config):
#     score_model.eval()
#     with torch.no_grad():
#         for batch in test_loader:
#             h, r, t = batch
#             h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])
#             t_emb = torch.tensor([0.0], dtype=torch.float, device=config["device"])
#             scores = score_model(h, r, t, t_emb)
            
#             # Compute metrics or perform ranking
#             # ...

# def main(config):
#     score_model, optimizer, train_loader, val_loader, test_loader = build_model(config)

#     for epoch in range(config["num_epochs"]):
#         train_loss = train_model(score_model, optimizer, train_loader, config)
#         test_loss = test_model(score_model, test_loader, config)
#         print(f"\n\nEpoch {epoch + 1}\nTrain Loss: {train_loss:.20f}\nTest Loss: {test_loss:.20f}")

#     compute_metrics(score_model, test_loader)

# config = {
#     "batch_size": 64,
#     "num_epochs": 2,
#     "num_steps": 1000,
#     "lr": 1e-4,
#     "device": device,
#     "noise_max": 0.5  # 20% of the original data values
# }

# main(config)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.nn import RotatE
from torch_geometric.datasets import FB15k_237
import sys
import os
import os.path as osp
import math
import numpy as np
from icecream import ic
from torchdiffeq import odeint  # library for solving differential equations

# Set up paths
parent_dir = os.path.abspath(os.pardir)
sys.path.append(parent_dir)
data_path = osp.join(parent_dir, "data", "FB15k_237")

# Load dataset
device = "cuda" if torch.cuda.is_available() else "cpu"
train_data = FB15k_237(data_path, split="train")[0].to(device)
val_data = FB15k_237(data_path, split="val")[0].to(device)
test_data = FB15k_237(data_path, split="test")[0].to(device)


class ScoreModel(nn.Module):
    def __init__(self, embedding_model, embedding_dim, relation_dim, config):
        super(ScoreModel, self).__init__()
        self.embedding_model = embedding_model
        self.embedding_dim = embedding_dim
        self.relation_dim = relation_dim
        self.config = config  # store config for hyperparameters

        self.score_net = nn.Sequential(
            nn.Linear(embedding_dim + relation_dim, 512, dtype=torch.float),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.float),
            nn.ReLU(),
            nn.Linear(512, 1, dtype=torch.float),
        )

    def forward(self, h, r, t, timestep=None):
        h_emb = self.embedding_model.node_emb(h)
        r_emb = self.embedding_model.rel_emb(r)
        t_emb = self.embedding_model.node_emb(t)

        # Implement your desired distance measure here (e.g., L2 distance)
        distance = torch.linalg.norm(h_emb + r_emb - t_emb, dim=-1)

        # Gradually increase the weight of the distance term during SDE steps
        weight = 0.0  # No weight increase if timestep is None
        if timestep is not None:
            def sigmoid(x):
                return 1 / (1 + math.exp(-x))
            weight = sigmoid(timestep / (self.config["num_steps"] - 1))
        score = weight * distance
        return score


def sde_func(t, y, h, r, score_model):
    """
    Function for the reverse SDE process.
    """
    t_emb = y.clone()  # current state of the target entity embedding
    with torch.no_grad():
        score_grad = torch.autograd.grad(score_model(h, r, t_emb, t), [t_emb])[0]
    return -score_grad  # Update in the direction that minimizes the score


def reverse_sde(h, r, t_init, score_model, config):
    """
    Perform reverse SDE to refine the target entity embedding.
    """
    t_steps = torch.linspace(0, 1.0, config["num_steps"]).to(config["device"])
    t_emb = odeint(sde_func, t_init, t_steps, h=h, r=r, score_model=score_model)[-1, :]
    return t_emb


def build_model(config):
    # Load pre-trained RotatE model (embedding_model)
    embedding_model = RotatE(num_nodes=train_data.num_nodes, num_relations=train_data.num_edge_types, hidden_channels=50).to(config["device"])
    embedding_model.load_state_dict(torch.load("../embedding_model/FB15k_237_RotatE_embedding_model_weights.pth"))
    embedding_model.eval()

    # Get entity and relation embeddings
    with torch.no_grad():
        entity_embeddings_real = embedding_model.node_emb.weight.detach()
        entity_embeddings_im = embedding_model.node_emb_im.weight.detach()
        entity_embeddings = torch.cat([entity_embeddings_real, entity_embeddings_im], dim=-1)
        relation_embeddings = embedding_model.rel_emb.weight.detach()

    # Initialize score-based model
    score_model = ScoreModel(embedding_model, entity_embeddings.shape[-1], relation_embeddings.shape[-1], config).to(config["device"])


    # Create data loaders
    train_dataset = TensorDataset(train_data.edge_index[0], train_data.edge_type, train_data.edge_index[1])
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)

    val_dataset = TensorDataset(val_data.edge_index[0], val_data.edge_type, val_data.edge_index[1])
    val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

    test_dataset = TensorDataset(test_data.edge_index[0], test_data.edge_type, test_data.edge_index[1])
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    # Initialize optimizer
    optimizer = Adam(score_model.parameters(), lr=config["lr"])


    return score_model, optimizer, train_loader, val_loader, test_loader


# Define loss function
def denoising_score_matching_loss(score_model, h, r, t, timestep):
    """
    Denoising score-matching loss with noise-conditional score networks.
    """
    true_score = score_model(h, r, t)  # simply do not pass in the timestep
    noisy_score = score_model(h, r, t, timestep)
    return ((true_score - noisy_score) ** 2).mean()


def train_model(score_model, optimizer, train_loader, config):
    score_model.train()
    loss_epoch = 0
    for batch in train_loader:
        h, r, t = batch
        h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])

        optimizer.zero_grad()
        for timestep in range(config["num_steps"]):
            loss = denoising_score_matching_loss(score_model, h, r, t, timestep)
            loss.backward()
        optimizer.step()

        loss_epoch += loss.item()

    return loss_epoch / len(train_loader)



def test_model(score_model, test_loader, config):
    score_model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            h, r, t = batch
            h, r, t = h.to(config["device"]), r.to(config["device"]), t.to(config["device"])
            for timestep in range(config["num_steps"]):
                loss = denoising_score_matching_loss(score_model, h, r, t, timestep)
                test_loss += loss.item()

    return test_loss / (len(test_loader) * config["num_steps"])  # Average across steps


def compute_metrics(score_model, test_loader, config):
    """
    Compute link prediction metrics: mean rank, mean reciprocal rank, and Hits@k.
    """
    score_model.eval()
    mean_rank = 0.0
    mean_reciprocal_rank = 0.0
    hits_at_10 = 0
    hits_at_3 = 0

    with torch.no_grad():
        for batch in test_loader:
            h, r, t = batch
            h, r = h.to(config["device"]), r.to(config["device"])

            # Sample a set of candidate target entities (t')
            num_candidates = 100  # You can adjust this value
            candidate_t = torch.randint(0, score_model.embedding_model.num_nodes, size=(h.shape[0], num_candidates)).to(config["device"])

            # Refine candidate embeddings using reverse SDE
            refined_candidates = []
            for i in range(h.shape[0]):
                t_init = score_model.embedding_model.node_emb(candidate_t[i])
                refined_candidate = reverse_sde(h[i], r[i], t_init, score_model, config)
                refined_candidates.append(refined_candidate)
            refined_candidates = torch.stack(refined_candidates, dim=0)

            # Compute scores for all candidates (including ground truth)
            all_t = torch.cat([t.unsqueeze(1), refined_candidates], dim=1)
            scores = score_model(h.repeat(1, num_candidates + 1), r.repeat(1, num_candidates + 1), all_t, torch.ones(h.shape[0], num_candidates + 1) * config["num_steps"]).squeeze(2)

            # Evaluate link prediction metrics
            for i in range(h.shape[0]):
                ground_truth_rank = (scores[i] == scores[i].max()).nonzero(as_tuple=True)[0].item()
                mean_rank += 1 + ground_truth_rank
                mean_reciprocal_rank += 1.0 / (1 + ground_truth_rank)
                hits_at_10 += (scores[i][:10] == scores[i].max()).any().item()
                hits_at_3 += (scores[i][:3] == scores[i].max()).any().item()

    mean_rank /= len(test_loader.dataset)
    mean_reciprocal_rank /= len(test_loader.dataset)
    hits_at_10 /= len(test_loader.dataset)
    hits_at_3 /= len(test_loader.dataset)

    print(f"\nMean Rank: {mean_rank:.4f}")
    print(f"Mean Reciprocal Rank: {mean_reciprocal_rank:.4f}")
    print(f"Hits@10: {hits_at_10:.4f}")
    print(f"Hits@3: {hits_at_3:.4f}")


def main(config):
    score_model, optimizer, train_loader, val_loader, test_loader = build_model(config)

    for epoch in range(config["num_epochs"]):
        train_loss = train_model(score_model, optimizer, train_loader, config)
        test_loss = test_model(score_model, test_loader, config)
        print(f"\n\nEpoch {epoch + 1}\nTrain Loss: {train_loss:.20f}\nTest Loss: {test_loss:.20f}")

    compute_metrics(score_model, test_loader, config)

config = {
    "batch_size": 64,
    "num_epochs": 1,
    "num_steps": 20,  # Adjust based on your desired SDE iterations
    "lr": 1e-4,
    "device": device,
}

main(config)



/home/blaineh2/anaconda3/envs/ScoreMatchingDiffKG/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0




Epoch 1
Train Loss: 21.83444665404613260762
Test Loss: 12.62340334191918422846


Epoch 2
Train Loss: 18.41383945504562547058
Test Loss: 10.42930970609188001674


TypeError: odeint() got an unexpected keyword argument 'args'